## Занятие 1. Обратное распространение ошибки + Numpy туториал. 

### Запуск тестов пакета

In [1]:
! pytest --pyargs abbyy_course_cvdl_t1.tests.test_relu

============================= test session starts =============================
platform win32 -- Python 3.7.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: D:\Repositories\abbyy_course_cvdl\lessons\l1
plugins: anyio-3.0.1, typeguard-2.7.1
collected 4 items

. FFFF                                                                   [100%]

================================== FAILURES ===================================
_______________________ TestLayer.test_init[test_data0] _______________________

self = <abbyy_course_cvdl_t1.tests.base.TestLayer object at 0x000001A1F8B15F88>
test_data = {'meta': {'description': 'The only ReLU test.', 'layer_kwargs': {}}, 'values': {'input': [[[[0.3445909917354584, -0.44...36354065], [0.320019006729126, 0.7765920162200928, 0.8497490286827087, 0.796563982963562, 0.9298689961433411]]]], ...}}
test_layer_cls = <class 'abbyy_course_cvdl_t1.relu.ReluLayer'>

    def test_init(self, test_data, test_layer_cls):
        values = test_data['values']
        me

In [2]:
import numpy as np
from abbyy_course_cvdl_t1.relu import ReluLayer
print(help(ReluLayer))

Help on class ReluLayer in module abbyy_course_cvdl_t1.relu:

class ReluLayer(abbyy_course_cvdl_t1.base.BaseLayer)
 |  Слой, выполняющий Relu активацию y = max(x, 0).
 |  Не имеет параметров.
 |  
 |  Method resolution order:
 |      ReluLayer
 |      abbyy_course_cvdl_t1.base.BaseLayer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  backward(self, output_grad: numpy.ndarray) -> numpy.ndarray
 |      Обратный проход, принимающий градиент ошибки по выходному тензору,
 |       (результату метода .forward) и возвращающий градиент ошибки по
 |       входному тензору (аргумент метода .forward).
 |      Метод .backward всегда вызывается после одного вызова .forward.
 |      Все слои всегда принимают и возвращают только один тензор.
 |  
 |  forward(self, input: numpy.ndarray) -> numpy.ndarray
 |      Прямой проход, выполняющий операцию над входными данными.
 |      Все слои всегда п

### Исправляем ReluLayer
Задача: дописать ReluLayer.\_\_init\_\_, .forward()

### Создание тензоров

In [3]:
a = np.random.rand(2, 3, 4) - 0.5
b = np.zeros(a.shape)
print(a) 
print(b)

[[[ 0.28750488 -0.33756362 -0.341148    0.48948753]
  [-0.47151425  0.35885633 -0.0084212   0.13203471]
  [-0.22166165  0.2084746   0.09357614 -0.15488479]]

 [[ 0.41247585  0.21567365  0.32508373  0.38682792]
  [-0.32251743  0.26023734 -0.1231991  -0.38744072]
  [-0.12572699  0.24214309  0.01106627 -0.40835472]]]
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


### Поиск максимума

In [4]:
np.max(a)

0.4894875330846026

In [5]:
np.maximum(a, b)

array([[[0.28750488, 0.        , 0.        , 0.48948753],
        [0.        , 0.35885633, 0.        , 0.13203471],
        [0.        , 0.2084746 , 0.09357614, 0.        ]],

       [[0.41247585, 0.21567365, 0.32508373, 0.38682792],
        [0.        , 0.26023734, 0.        , 0.        ],
        [0.        , 0.24214309, 0.01106627, 0.        ]]])

In [6]:
np.argmax(a)

3

In [7]:
print(a.flatten()[np.argmax(a)])
print('---IndexError----')
a[np.argmax(a)]

0.4894875330846026
---IndexError----


IndexError: index 3 is out of bounds for axis 0 with size 2

### Поэлементные операции

In [8]:
a * b

array([[[ 0., -0., -0.,  0.],
        [-0.,  0., -0.,  0.],
        [-0.,  0.,  0., -0.]],

       [[ 0.,  0.,  0.,  0.],
        [-0.,  0., -0., -0.],
        [-0.,  0.,  0., -0.]]])

In [9]:
a + b

array([[[ 0.28750488, -0.33756362, -0.341148  ,  0.48948753],
        [-0.47151425,  0.35885633, -0.0084212 ,  0.13203471],
        [-0.22166165,  0.2084746 ,  0.09357614, -0.15488479]],

       [[ 0.41247585,  0.21567365,  0.32508373,  0.38682792],
        [-0.32251743,  0.26023734, -0.1231991 , -0.38744072],
        [-0.12572699,  0.24214309,  0.01106627, -0.40835472]]])

### Индексация

In [10]:
print("a             \t", a.shape)
print("a[0]          \t", a[0].shape)
print("a[:, 0]       \t", a[:, 0].shape)
print("a[:, 0, 1:2]  \t", a[:, 0, 0:2].shape)
print("a[:, [0], 1:2]\t", a[:, [0], 0:2].shape)
print("a[:, None]    \t", a[:, None].shape)

a             	 (2, 3, 4)
a[0]          	 (3, 4)
a[:, 0]       	 (2, 4)
a[:, 0, 1:2]  	 (2, 2)
a[:, [0], 1:2]	 (2, 1, 2)
a[:, None]    	 (2, 1, 3, 4)


In [11]:
idx = a > 0
print(a.shape, idx.shape, idx.dtype)

(2, 3, 4) (2, 3, 4) bool


In [12]:
a[idx].shape

(13,)

In [13]:
a[a == np.max(a)]

array([0.48948753])

In [14]:
c = np.arange(
    np.prod(a.shape)
).reshape(a.shape)
print(c.shape)
c

(2, 3, 4)


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [15]:
c[
    [0, 1, 0, 0,],
    [1, -1, 2, -1],
    [3, 2, 1, 0]
]

array([ 7, 22,  9,  8])

In [16]:
c[
    [[0, 1],[0, 0,]],
    [[1, -1],[2, -1]],
    [[3, 2],[1, 0,]],
]

array([[ 7, 22],
       [ 9,  8]])

### Исправляем ReluLayer
Задача: дописать ReluLayer.backward()

In [18]:
! pytest --pyargs abbyy_course_cvdl_t1.tests.test_relu -x

============================= test session starts =============================
platform win32 -- Python 3.7.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: D:\Repositories\abbyy_course_cvdl\lessons\l1
plugins: anyio-3.0.1, typeguard-2.7.1
collected 4 items

. ....                                                                   [100%]

============================== 4 passed in 0.10s ==============================


### Broadcasting

In [19]:
a = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
])
b = np.array([[10, 20]])
print(a.shape, b.shape, (a*b).shape)
print(a * b)

(4, 2) (1, 2) (4, 2)
[[  0  20]
 [ 20  60]
 [ 40 100]
 [ 60 140]]


In [20]:
b = np.array([10, 20, 30, 40])
print(a.shape, b.shape)
print("Broadcast error")
print("---------------")
a * b


(4, 2) (4,)
Broadcast error
---------------


ValueError: operands could not be broadcast together with shapes (4,2) (4,) 

In [21]:
a * b[:, None]

array([[  0,  10],
       [ 40,  60],
       [120, 150],
       [240, 280]])

In [22]:
a = np.array([
    [0, 1],
    [1, 0],
    [1, 1]
])
b = np.array([
    [10, 20],
    [30, 40],
    [50, 60],
    [70, 80]
])
c = a[None] * b[:, None]
print("c[i, j, k] = a[1, i, k] * b[j, 1, k]")
print("------------------------------")
print(a.shape, b.shape, c.shape)

print(c)

c[i, j, k] = a[1, i, k] * b[j, 1, k]
------------------------------
(3, 2) (4, 2) (4, 3, 2)
[[[ 0 20]
  [10  0]
  [10 20]]

 [[ 0 40]
  [30  0]
  [30 40]]

 [[ 0 60]
  [50  0]
  [50 60]]

 [[ 0 80]
  [70  0]
  [70 80]]]


### Tensordot

In [23]:
help(np.tensordot)

Help on function tensordot in module numpy:

tensordot(a, b, axes=2)
    Compute tensor dot product along specified axes.
    
    Given two tensors, `a` and `b`, and an array_like object containing
    two array_like objects, ``(a_axes, b_axes)``, sum the products of
    `a`'s and `b`'s elements (components) over the axes specified by
    ``a_axes`` and ``b_axes``. The third argument can be a single non-negative
    integer_like scalar, ``N``; if it is such, then the last ``N`` dimensions
    of `a` and the first ``N`` dimensions of `b` are summed over.
    
    Parameters
    ----------
    a, b : array_like
        Tensors to "dot".
    
    axes : int or (2,) array_like
        * integer_like
          If an int N, sum over the last N axes of `a` and the first N axes
          of `b` in order. The sizes of the corresponding axes must match.
        * (2,) array_like
          Or, a list of axes to be summed over, first sequence applying to `a`,
          second to `b`. Both element

In [24]:
a = np.array([
    [0.5, -0.5],
    [-0.5, 0.5],
    [1, 0.]
])
b = np.array([10, 1])
print("c[i, k] = Sum(a[i, j] * b[j, k]")
print('-------------------------------')
print(a.shape, b.shape)
a @ b

c[i, k] = Sum(a[i, j] * b[j, k]
-------------------------------
(3, 2) (2,)


array([ 4.5, -4.5, 10. ])

In [25]:
np.tensordot(
    a, b,
    axes=[1, 0]
)

array([ 4.5, -4.5, 10. ])

In [26]:
a = np.arange(2* 3 *4).reshape((2, 3 ,4))
a

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [27]:
b = np.arange(4 * 2).reshape((4, 2)) % 2
b

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1]], dtype=int32)

In [28]:
np.tensordot(
    a, b,
    axes=[
        [-1, 0],
        [0, 1]
    ]
)

array([54, 70, 86])

### View vs Copy

In [29]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[1:, 1:, 1:]
b += 0.33
print(b)
print("---------")
print(a)

[[[17.33 18.33 19.33]
  [21.33 22.33 23.33]]]
---------
[[[ 0.    1.    2.    3.  ]
  [ 4.    5.    6.    7.  ]
  [ 8.    9.   10.   11.  ]]

 [[12.   13.   14.   15.  ]
  [16.   17.33 18.33 19.33]
  [20.   21.33 22.33 23.33]]]


In [30]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[1, 1, 1]
b += 0.33
print(b)
print("---------")
print(a)

17.33
---------
[[[ 0.  1.  2.  3.]
  [ 4.  5.  6.  7.]
  [ 8.  9. 10. 11.]]

 [[12. 13. 14. 15.]
  [16. 17. 18. 19.]
  [20. 21. 22. 23.]]]


In [31]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[None, 1:, 1:, 1:]
b += 0.33
print(b)
print("---------")
print(a)

[[[[17.33 18.33 19.33]
   [21.33 22.33 23.33]]]]
---------
[[[ 0.    1.    2.    3.  ]
  [ 4.    5.    6.    7.  ]
  [ 8.    9.   10.   11.  ]]

 [[12.   13.   14.   15.  ]
  [16.   17.33 18.33 19.33]
  [20.   21.33 22.33 23.33]]]


In [32]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
a[[0,0,0],[0, 1, 2], [-1, -1, -1]] += 0.33
print(a)

[[[ 0.    1.    2.    3.33]
  [ 4.    5.    6.    7.33]
  [ 8.    9.   10.   11.33]]

 [[12.   13.   14.   15.  ]
  [16.   17.   18.   19.  ]
  [20.   21.   22.   23.  ]]]


In [33]:
a = np.arange(2 * 3 * 4).reshape((2, 3 , 4)).astype(float)
b = a[[0,0,0],[0, 1, 2], [-1, -1, -1]] 
b += 0.33
print(a)

[[[ 0.  1.  2.  3.]
  [ 4.  5.  6.  7.]
  [ 8.  9. 10. 11.]]

 [[12. 13. 14. 15.]
  [16. 17. 18. 19.]
  [20. 21. 22. 23.]]]
